In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import requests
import pandas as pd

df = pd.read_csv("https://www.dropbox.com/scl/fi/o9x7u2sfwimnydlu8qreb/Imine-Reductase-Evolution.csv?rlkey=17kea1xtrihpvmfvtjsmxp3bm&dl=1", index_col=0).dropna()


In [ ]:
wt_sequence = 'MTNHASTATTSPAPVTVVGLGPMGLVLAEVLLAKGHPTTVWNRTPERASGLVAQGASLAASITDAVSASPVTIMCLNNYATMYEVFGPAREALRDRVLVNLNSGTPQEVRAAVSWASDLGTRYLDGAIMVPPPLVGRPDAVFLYSGDRAVLDEHRATLASLGDPRFLGADPTLAVLYNTALLHMMYATLNGYLQATALVGSAGVSATEFADIALGWFAPSVLAPSSLAAHAVDLDKGNYPGTLGTLRMNVNALEHIARAAEEQGVHSELPHLMREVAERAVAQGHGDHNYMSVYEAFKQPSPAS'


In [ ]:
from seqlike import aaSeqLike
from seqlike.MutationSet import MutationSet


mutation_sets = []
for mutation in df["mutation"]:
    try:
        if ";" in mutation:
            mutation_sets.append(MutationSet(mutations=mutation.split("; ")))
        else:
            mutation_sets.append(MutationSet(mutations=[mutation]))
    except Exception as e:
        print(mutation)
        print(e)
df = df.assign(**{"mutation_set": mutation_sets, "mutation_string": list(map(str, mutation_sets))})


In [ ]:
str(mutation_sets[0])


In [ ]:
wt_sequence = aaSeqLike('MTNHASTATTSPAPVTVVGLGPMGLVLAEVLLAKGHPTTVWNRTPERASGLVAQGASLAASITDAVSASPVTIMCLNNYATMYEVFGPAREALRDRVLVNLNSGTPQEVRAAVSWASDLGTRYLDGAIMVPPPLVGRPDAVFLYSGDRAVLDEHRATLASLGDPRFLGADPTLAVLYNTALLHMMYATLNGYLQATALVGSAGVSATEFADIALGWFAPSVLAPSSLAAHAVDLDKGNYPGTLGTLRMNVNALEHIARAAEEQGVHSELPHLMREVAERAVAQGHGDHNYMSVYEAFKQPSPAS')


In [ ]:
wt_sequence + mutation_sets[0]


In [ ]:
from tqdm.auto import tqdm

one_hots = []
valid_mutation_sets = []
for mutations in tqdm(mutation_sets):
    try:
        one_hots.append((wt_sequence + mutations).to_onehot())
        valid_mutation_sets.append(mutations)
    except Exception as e:
        pass


In [ ]:
len(one_hots), len(df)


In [ ]:
import jax.numpy as np
data = pd.DataFrame({"one_hot": one_hots, "mutation_set": valid_mutation_sets, "mutation_string": list(map(str, valid_mutation_sets))})
data = data.merge(df, on=["mutation_string"])


# genotypes = data["one_hot"].apply(np.array)
# phenotypes = data["mean"].apply(np.log)


In [ ]:
data["one_hot"].iloc[0].dtype


In [ ]:
genotypes = np.dstack([d for d in data["one_hot"].apply(np.array)]).transpose((2, 0, 1))
genotypes


In [ ]:
phenotypes = np.vstack(data["mean"].apply(np.log)).squeeze()
phenotypes


In [ ]:
from protein_reference_free_analysis.effects import zeroth_order_effects, first_order_effects, second_order_effects


e_0 = zeroth_order_effects(genotypes, phenotypes)


In [ ]:
e_1 = first_order_effects(genotypes, phenotypes)


In [ ]:
# This takes too long to compute on a real protein.
# I need to optimize the implementation so that we only look at positions that are varying
# and ignore other positions.
# Time to go back to looking at the source code again...
# e_2 = second_order_effects(genotypes, phenotypes)
